# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-02-28","2019-03-03",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-28 to 2019-03-03
536


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

100%|██████████| 321/321 [00:00<00:00, 4955.31it/s]

Binary Encoding


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

                                               title  \
0  'We must be saints,' young Catholic says about...   
1  The Daily 202: Michael Cohen hearing showcased...   
2  What YouTube needs to do to clean up its thorn...   
3  Trump-Kim summit ends abruptly with no deal: '...   
4  Business schools join forces to develop online...   

                                         description  
0  Being Catholic is one of the most unpopular th...  
1  Trump criticizes timing of ‘fake hearing’ afte...  
2  Google owned video network lost advertisers an...  
3  Plus, 'impressed' Trump slams 'fake' Michael C...  
4  Business schools join forces to develop online...  


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|██████████| 321/321 [01:14<00:00,  3.66it/s]


In [6]:
articleDB

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,context,tags,tags_top_5
0,0.225382,0.774618,https://www.cbc.ca/news/canada/manitoba/young-...,1,"'We must be saints,' young Catholic says about...",Being Catholic is one of the most unpopular th...,cbc-news,2019-03-03T12:00:00Z,abuse summit took place rome address global p...,"In February 2019, an abuse summit took place i...","In February 2019, an abuse summit took place i...","abuse summit, took, Rome, global problem, abus...","abuse summit, rome, global problem, abuse scan...","right thing,st. vital catholic church,difficul..."
1,0.234248,0.765752,https://www.washingtonpost.com/news/powerpost/...,1,The Daily 202: Michael Cohen hearing showcased...,Trump criticizes timing of ‘fake hearing’ afte...,the-washington-post,2019-02-28T14:06:39Z,big idea cutting short summit with north kore...,THE BIG IDEA: After cutting short his summit w...,THE BIG IDEA: After cutting short his summit w...,"BIG IDEA, short summit, long journey, Washingt...","big idea, short summit, long journey, washingt...","trump presidency,first instinct,presidential c..."
2,0.283077,0.716923,https://www.usatoday.com/story/tech/talkingtec...,1,What YouTube needs to do to clean up its thorn...,Google owned video network lost advertisers an...,usa-today,2019-03-02T14:15:29Z,network receives hours new content every minu...,But a network that receives 500 hours of new c...,But a network that receives 500 hours of new c...,"network, new content, seems, non stop game, wh...","network, new content, non stop game, whack mol...","specs,talking tech,notarize,smart home product..."
3,0.323753,0.676247,https://www.foxnews.com/us/trump-kim-summit-en...,1,Trump-Kim summit ends abruptly with no deal: '...,"Plus, 'impressed' Trump slams 'fake' Michael C...",fox-news,2019-02-28T10:06:42Z,trump summit fell north demanded full removal ...,Trump said the summit fell through after the N...,Trump said the summit fell through after the N...,"Trump, summit, North, full removal, internatio...","trump, summit, north, full removal, internatio...","sen. john cornyn,mike pompeo,black students,de..."
4,0.358581,0.641419,https://www.ft.com/content/3befa2d8-1824-11e9-...,1,Business schools join forces to develop online...,Business schools join forces to develop online...,financial-times,2019-03-03T20:01:00Z,teamwork core element learning mba courses bus...,Teamwork is a core element of learning on MBA ...,Teamwork is a core element of learning on MBA ...,"Teamwork, MBA courses, business schools, pract...","teamwork, mba courses, business schools, new w...","lee kong chian school,digital education,course..."
5,0.393527,0.606473,https://business.financialpost.com/investing/t...,1,Bill Gross says this secret condition made him...,Even after one of the most storied careers in ...,financial-post,2019-03-01T19:46:01Z,even one storied careers financial markets bil...,Even after one of the most storied careers in ...,Even after one of the most storied careers in ...,"storied careers, financial markets, Bill Gross...","storied careers, financial markets, bill gross...","preference,been bond king,crown,low rate polic..."
6,0.413834,0.586166,https://www.economist.com/books-and-arts/2019/...,1,What to look for in a usage and grammar guide,"“Dreyer’s English”, a bestselling book, is rar...",the-economist,2019-02-28T15:55:48Z,people buy books english usage obvious answer...,WHY DO PEOPLE buy books on English usage? The ...,WHY DO PEOPLE buy books on English usage? The ...,"PEOPLE, buy, English usage, obvious answer, au...","english usage, obvious answer, authoritative a...","columnist,language columnist,17th edition,oliv..."
7,0.436507,0.563493,https://www.foxnews.com/politics/from-reparati...,1,"From reparations to Green New Deal, liberal li...","From reparations to Green New Deal, liberal li...",fox-news,2019-03-01T19:16:41Z,democrats presidential hopefuls latching onto...,The Democrats' 2020 presidential hopefu

In [7]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:00<00:00, 568.21it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 321/321 [00:00<00:00, 541.03it/s]


         deal  president     trump  abruptly    walked      away  \
0    0.082353   0.116505  0.351852     0.125  0.142857  0.014925   
1    0.011765   0.000000  0.000000     0.000  0.000000  0.000000   
2    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
3    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
4    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
5    0.000000   0.009709  0.037037     0.000  0.000000  0.014925   
6    0.011765   0.000000  0.000000     0.000  0.000000  0.000000   
7    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
8    0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
9    0.035294   0.213592  1.629630     0.125  0.000000  0.014925   
10   0.000000   0.000000  0.000000     0.000  0.000000  0.000000   
11   0.000000   0.019417  0.240741     0.000  0.000000  0.014925   
12   0.000000   0.019417  0.185185     0.000  0.000000  0.014925   
13   0.000000   0.009709  0.000000     0.000  0.

[321 rows x 17990 columns]


In [8]:
articleDB.head()

,nonRel,Rel,url,prediction,title,description,source,date,content,origContent,contentWithStops,tags,tags_top_5,related_articles
0,0.133170,0.866830,https://www.foxnews.com/us/trump-kim-summit-en...,1,Trump-Kim summit ends abruptly with no deal: '...,"Plus, 'impressed' Trump slams 'fake' Michael C...",fox-news,2019-02-28T10:06:42Z,deal president trump abruptly walked away neg...,NO DEAL: President Trump abruptly walked away ...,deal president trump abruptly walked away neg...,"president trump, away negotiations, back washi...","personal lawyer,first lady,todd starnes,soon s...","46, 211, 9"
1,0.272690,0.727310,https://www.cbc.ca/news/canada/manitoba/young-...,1,"'We must be saints,' young Catholic says about...",Being Catholic is one of the most unpopular th...,cbc-news,2019-03-03T12:00:00Z,abuse summit took place rome address global p...,"In February 2019, an abuse summit took place i...","2019, abuse summit took place rome address gl...","abuse summit, catholic church, roland lanoie, ...","entirely objective,unpopular things,great mill...","163, 75, 35"
2,0.358581,0.641419,https://www.ft.com/content/3befa2d8-1824-11e9-...,1,Business schools join forces to develop online...,Business schools join forces to develop online...,financial-times,2019-03-03T20:01:00Z,teamwork core element learning mba courses bus...,Teamwork is a core element of learning on MBA ...,"teamwork core element learning mba courses. , ...","new ways, online courses, defensive step, esta...","organisations course,global platform,mr vanden...","77, 282, 113"
3,0.375336,0.624664,https://www.economist.com/books-and-arts/2019/...,1,What to look for in a usage and grammar guide,"“Dreyer’s English”, a bestselling book, is rar...",the-economist,2019-02-28T15:55:48Z,people buy books english usage obvious answer...,WHY DO PEOPLE buy books on English usage? The ...,people buy books english usage? obvious answe...,"recent edition, small pagesnor, ms truss, dail...","recent edition,month america,serious stuff,inf...","226, 28, 111"
4,0.386786,0.613214,https://www.economist.com/finance-and-economic...,1,New data sources make it harder for executives...,How to cross-check Elon Musk,the-economist,2019-02-28T15:58:13Z,go straight useful rule anyone seeking accurat...,GO STRAIGHT to the source is a useful rule for...,go straight useful rule anyone seeking accurat...,"caused firms, false sec, teslas chairman, chie...","teslas chairman,condition 20th,false sec,inter...","73, 309, 197"


In [9]:
import frontPage as fp
frontpage, artDict, rel_arts = fp.FrontPage(articleDB, trendingTermsDB)